In [1]:
import numpy as np
import random
import pickle
import time

In [4]:
learning_rate=0.01 #alpha
decay_gamma=0.7 #gamma
exp_rate=0.3 #epsilon
starting_player='agent'
list_of_winners=[]
policy={}
initial_board=[3,5]

In [14]:
class NimGame:
    def __init__(self, init_board):
        self.board=initial_board.copy()
        self.number_of_piles=np.count_nonzero(self.board)
        self.player=starting_player
        self.winner=None
        self.starting_player=starting_player
        self.states_value={}
        self.states=[]
        self.exp_rate=exp_rate
        self.lr=learning_rate
        self.gamma=decay_gamma
    def change_player(self):
        """A function for changing player"""
        if self.player=='agent':
            self.player='dummy'
        else:
            self.player='agent'
    def get_reward(self):
        """This function determines the reward to give.
            Returns the reward."""
        if self.player =='agent':
            if sum(self.board)==1:
                return 1 
        if self.is_game_over():
            if self.player == 'agent': #if player 1 takes last stick
                return -1
            else:
                return 1 # in player 2 takes last stick
        else:
            return 0 #if game is not over
    def get_legal_actions(self):
        """This function determines all legal actions on the current board state.
            Returns a list of legal actions."""
        actions = []
        for i in np.nonzero(self.board)[0]:#indices of non-zero piles
            for j in range(1, self.board[i]+1): #number of stones
                actions.append((i, j))
        return actions
    def is_game_over(self):
        """A function for determining whether the game is over.
            Returns True if the game is over and False otherwise"""
        return sum(self.board)==0
    def possible_next_states(self, state):
        """Function to consider all possible board states which is obtainable from a given state.
            Input is a board state for which you want to find all possible obtainable states from.
            Returns a list of all possible obtainable board states."""
        list_to_return=[]
        for i in range(state[0]+1):
            for j in range(state[1]+1):
                list_to_return.append([state[0]-i, state[1]-j])
        if state in list_to_return:
            list_to_return.remove(state)
        return list_to_return
    def feedReward(self, reward):
        """Function for updating the Q-table"""
        for st in reversed(self.states):  # goes through all saved board states of this game
            possible_next_st=self.possible_next_states(st)
            temp=-1000
            for i in possible_next_st:
                if self.states_value.get(str(i)) is not None and self.states_value.get(str(i))>temp:
                    temp=self.states_value.get(str(i))
            if self.states_value.get(str(st)) is None: 
                self.states_value[str(st)] = 0  # initialise a value for the state
            self.states_value[str(st)] += self.lr * (reward -self.gamma*temp- self.states_value[str(st)])
    def reset(self,init_state):
        """A function for resetting the game after an ended game."""
        self.board=init_state
        self.states=[]
        self.number_of_piles=np.count_nonzero(self.board)
        self.player=self.starting_player
        self.winner=None
        return
    def nim_sum(self, numbers):
        """Function for calculating the nim sum.
            Input is a list of numbers.
            Returns Nim sum of the numbers in the list."""
        binary_numbers = [bin(num)[2:].zfill(len(bin(max(numbers))) - 2) for num in numbers]
        column_sums = [sum(int(binary[i]) for binary in binary_numbers) for i in range(len(binary_numbers[0]))]
        nim_sum = ''.join(['0' if sum % 2 == 0 else '1' for sum in column_sums])
        return int(nim_sum, 2)
    def choose_optimal_action(self):
        """This function returns the optimal action, i.e. such that the nim sum is zero.
        if this is not possible, then it returns a random action"""
        if self.nim_sum(self.board)==0:
            return self.choose_random_action()
        else:
            actions=self.get_legal_actions()
            for a in actions:
                test_board=self.board.copy()
                pile_number, amount_to_remove=a
                test_board[pile_number]-=amount_to_remove
                if self.nim_sum(test_board)==0:
                    return a
    def choose_random_action(self):
        """A function for choosing an action randomly.
            Returns an action."""
        action=self.get_legal_actions()
        return random.choice(action)
    def choose_smart_action(self):
        """A function for choosing an action according to the Q-table.
        Returns an action."""
        actions=self.get_legal_actions()
        value_max = -999
        for p in actions:
            board_copy=self.board.copy()
            pile_number, amount_to_remove=p
            board_copy[pile_number]-=amount_to_remove
            next_board = board_copy
            value = 0 if self.states_value.get(str(next_board)) is None else self.states_value.get(str(next_board))
            if value >= value_max:
                value_max = value
                move = p
        return move
    def play_action(self, action):
        """play a given action."""
        pile_number, amount_to_remove=action
        self.board[pile_number]-=amount_to_remove
        return
    def training_game(self, rounds=10):
        """The function for training the agent.
            Input is the number of training games to be played"""
        for i in range(rounds):
            while not self.is_game_over():
                if self.player=='dummy':
                    move= self.choose_random_action()
                else:
                    if np.random.uniform(0,1)<= self.exp_rate:
                        move = self.choose_random_action()
                    else:
                        move=self.choose_smart_action()
                self.play_action(move)
                if self.player=='agent':
                    self.states.append(self.board.copy())
                if self.is_game_over():
                    self.winner=self.player
                    list_of_winners.append(self.winner)
                    if self.winner=='agent':
                        self.feedReward(5)
                    else:
                        self.feedReward(-1)
                    self.reset(initial_board.copy())
                    break
                else:
                    self.change_player()
    def train_against_pro(self, rounds=1000):
        """In this function, the agent is trained against a player who only performs optimal moves
        - note that this player is still named dummy.
            Input is the number of training games to be played."""
        for i in range(rounds):
            while not self.is_game_over():
                if self.player=='dummy':
                    move= self.choose_optimal_action()
                else:
                    if np.random.uniform(0,1)<= self.exp_rate:
                        move = self.choose_random_action()
                    else:
                        move=self.choose_smart_action()
                self.play_action(move)
                if self.player=='agent':
                    self.states.append(self.board.copy())
                    #print(self.board, self.states, 'hej')
                if self.is_game_over():
                    self.winner=self.player
                    list_of_winners.append(self.winner)
                    if self.winner=='agent':
                        self.feedReward(100)
                    else:
                        self.feedReward(-1)
                    self.reset(initial_board.copy())
                    break
                else:
                    self.change_player()
    def test_against_pro(self, rounds=1000):
        """A function for testing a given policy against a player performing optimal moves.
            Input is the number of test games to be played."""
        for i in range(rounds):
            while not self.is_game_over():
                if self.player=='dummy':
                    move= self.choose_optimal_action()
                    #print('dummy chooses the following move', move)
                else:
                        move=self.choose_smart_action()
                        #print('agent chooses the following move', move)
                self.play_action(move)
                #print('the board is now:', self.board, 'with nim sum:', self.nim_sum(self.board))
                if self.is_game_over():
                    self.winner=self.player
                    list_of_winners.append(self.winner)
                    self.reset(initial_board.copy())
                    break
                else:
                    self.change_player()
    def test_game(self, rounds=1000):
        """A function for testing a policy against a player with the random strategy.
            Input is the number of test game to be played."""
        for i in range(rounds):
            while not self.is_game_over():
                if self.player=='dummy':
                    move= self.choose_random_action()
                else:
                        move=self.choose_smart_action()
                self.play_action(move)
                if self.is_game_over():
                    self.winner=self.player
                    list_of_winners.append(self.winner)
                    self.reset(initial_board.copy())
                    break
                else:
                    self.change_player()


In [15]:
nim=NimGame(initial_board)

## Below is to find the mean number of training games and mean number of seconds to find an optimal policy, when training and testing against a player using the random strategy

In [9]:
number_of_iterations=20
max_win_rate=0
rounds_mean=[]
time_mean=[]
for i in range(number_of_iterations):
    print(i)
    list_of_winners=[]
    policy={}
    nim.states_value=policy
    total_rounds=0
    rounds=1000
    start_time=time.time()
    while list_of_winners.count('agent')*100/rounds<100:
        list_of_winners=[]
        nim.training_game(rounds)
        list_of_winners=[]
        nim.test_game(rounds)
        total_rounds+=rounds
        #print('agent wins:', list_of_winners.count('agent')*100/rounds, total_rounds)
        if list_of_winners.count('agent')*100/rounds>max_win_rate:
            max_win_rate=list_of_winners.count('agent')*100/rounds
    end_time=time.time()
    rounds_mean.append(total_rounds)
    time_mean.append(end_time-start_time)
    #print(total_rounds, end_time-start_time)
print('In average, it took', np.mean(rounds_mean),'training games and', np.mean(time_mean), 'seconds to find an optimal policy.')

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
In average, it took 17150.0 training games and 1.539172852039337 seconds to find an optimal policy.


### The cell below allows to test the last policy obtained in the cell above.

In [838]:
list_of_winners=[]
nim.test_game(rounds)
print('agent wins:', list_of_winners.count('agent')*100/rounds, '% of the games against a player with the random strategy')
list_of_winners=[]
nim.test_against_pro(rounds)
print('agent wins:', list_of_winners.count('agent')*100/rounds, '% of the games against a player with the optimal strategy')

agent wins: 100.0 % of the games against a player with the random strategy
agent wins: 100.0 % of the games against a player with the optimal strategy


## Below is to find the mean number of training games and mean number of seconds to find an optimal policy, when training against a player using the random strategy and testing against a player with the optimal strategy

In [16]:
number_of_iterations=20
rounds_mean=[]
time_mean=[]
for i in range(number_of_iterations):
    list_of_winners=[]
    policy={}
    nim.states_value=policy
    total_rounds=0
    rounds=1000
    start_time=time.time()
    while list_of_winners.count('agent')*100/rounds<100:
        list_of_winners=[]
        nim.training_game(rounds)
        list_of_winners=[]
        nim.test_against_pro(rounds)
        total_rounds+=rounds
        #print('agent wins:', list_of_winners.count('agent')*100/rounds, total_rounds)
        if list_of_winners.count('agent')*100/rounds>max_win_rate:
            max_win_rate=list_of_winners.count('agent')*100/rounds
    end_time=time.time()
    rounds_mean.append(total_rounds)
    time_mean.append(end_time-start_time)
    #print(total_rounds, end_time-start_time)
print('In average, it took', np.mean(rounds_mean),'training games and', np.mean(time_mean), 'seconds to find an optimal policy.')

In average, it took 19050.0 training games and 2.182584536075592 seconds to find an optimal policy.


## Below is to find the mean number of training games and mean number of seconds to find an optimal policy, when training and testing against a player using the optimal strategy

In [895]:
number_of_iterations=20
rounds_mean=[]
time_mean=[]
for i in range(number_of_iterations):
    list_of_winners=[]
    policy={}
    nim.states_value=policy
    total_rounds=0
    rounds=1000
    start_time=time.time()
    while list_of_winners.count('agent')*100/rounds<100:
        list_of_winners=[]
        nim.train_against_pro(rounds)
        list_of_winners=[]
        nim.test_against_pro(rounds)
        total_rounds+=rounds
        #print('agent wins:', list_of_winners.count('agent')*100/rounds, total_rounds)
        if list_of_winners.count('agent')*100/rounds>max_win_rate:
            max_win_rate=list_of_winners.count('agent')*100/rounds
    end_time=time.time()
    rounds_mean.append(total_rounds)
    time_mean.append(end_time-start_time)
    #print(total_rounds, end_time-start_time)
print('In average, it took', np.mean(rounds_mean),'training games and', np.mean(time_mean), 'seconds to find an optimal policy.')

In average, it took 1000.0 training games and 0.10912256240844727 seconds to find an optimal policy.


In [872]:
policy

{'[3, 0]': 0.26382104116971833,
 '[1, 0]': 5.7162118005828155,
 '[1, 3]': -0.04360312565677414,
 '[3, 4]': -0.3989787047473254,
 '[3, 1]': -0.49607643868958107,
 '[1, 5]': -0.27153024578615564,
 '[0, 5]': 2.030846617443289,
 '[0, 1]': 2.1879985124615615,
 '[2, 1]': -0.2794442255918559,
 '[3, 3]': 11.403877974899833,
 '[0, 2]': -0.2571038250326191,
 '[2, 5]': -0.31277553712234285,
 '[1, 2]': -0.2104838570811754,
 '[3, 2]': -0.3572855407835115,
 '[2, 0]': -0.24407521440403018,
 '[0, 0]': 27.988329071742296,
 '[1, 1]': 9.43119351756959,
 '[2, 2]': 1.819429520442433,
 '[0, 3]': -0.07769159369677414,
 '[2, 3]': -0.036250837446204345}

## Below is to find the highest values boards in the policy found above

In [842]:
import ast

my_dict=nim.states_value
sorted_dict = sorted(my_dict.items(), key=lambda x: x[1], reverse=True)
top_keys = [x[0] for x in sorted_dict[:15]]

best_boards=top_keys
for i in best_boards:
    print( i,my_dict[str(i)],nim_sum(ast.literal_eval(i)))

[0, 0] 392.01140300034746 0
[1, 1] -18.70541873037432 0
[2, 2] -33.994012623969816 0
[2, 3] -34.19266400944704 1
[1, 3] -34.673776287668886 2
[0, 3] -36.37541362992991 3
[1, 2] -59.61555056478125 3
[0, 2] -60.65206542224638 2
[2, 0] -61.560749291072895 2
[2, 1] -62.479975528018926 3
[3, 3] -88.15937832122076 0
[0, 5] -88.42828081616597 5
[3, 2] -88.48862702675608 1
[3, 4] -89.49960824655686 7
[2, 5] -89.6334055349269 7
